<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Example-Together" data-toc-modified-id="Example-Together-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example Together</a></span><ul class="toc-item"><li><span><a href="#Question" data-toc-modified-id="Question-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Question</a></span></li><li><span><a href="#Considerations" data-toc-modified-id="Considerations-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Considerations</a></span></li><li><span><a href="#Loading-the-Data" data-toc-modified-id="Loading-the-Data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Loading the Data</a></span></li><li><span><a href="#Some-Exploration-to-Better-Understand-our-Data" data-toc-modified-id="Some-Exploration-to-Better-Understand-our-Data-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Some Exploration to Better Understand our Data</a></span></li><li><span><a href="#Experimental-Setup" data-toc-modified-id="Experimental-Setup-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Experimental Setup</a></span><ul class="toc-item"><li><span><a href="#What-Test-Would-Make-Sense?" data-toc-modified-id="What-Test-Would-Make-Sense?-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>What Test Would Make Sense?</a></span></li><li><span><a href="#The-Hypotheses" data-toc-modified-id="The-Hypotheses-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>The Hypotheses</a></span></li><li><span><a href="#Setting-a-Threshold" data-toc-modified-id="Setting-a-Threshold-2.5.3"><span class="toc-item-num">2.5.3&nbsp;&nbsp;</span>Setting a Threshold</a></span></li></ul></li><li><span><a href="#$\chi^2$-Test" data-toc-modified-id="$\chi^2$-Test-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>$\chi^2$ Test</a></span><ul class="toc-item"><li><span><a href="#Setup-the-Data" data-toc-modified-id="Setup-the-Data-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Setup the Data</a></span></li><li><span><a href="#Calculation" data-toc-modified-id="Calculation-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Calculation</a></span></li></ul></li><li><span><a href="#Interpretation" data-toc-modified-id="Interpretation-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Interpretation</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></div>

In [36]:
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

# Objectives

- Conduct an A/B test in Python
- Interpret the results of the A/B tests for a stakeholder

# Example Together

## Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

## Considerations

What would we need to consider when designing our experiment?

Might include:

- Who is it that we're including in our test?
- How big of an effect would make it "worth" us seeing?
    - This can affect sample size
    - This can give context of a statistically significant result
- Other biases or "gotchas"

## Loading the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics) via the release page of this repo: https://github.com/flatiron-school/ds-ab_testing/releases 

The code below will load it into our DataFrame:

In [ ]:
# This will download the data from online so it can take
# some time (but relatively small download)

df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/\
releases/download/v1.2/products_small.csv')

> Let's take a look while we're at it

In [ ]:
# Check it out
df.head()

In [ ]:
# All the deets
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

## Some Exploration to Better Understand our Data

Lets's look at the different banner types:

In [ ]:
# banner types

In [ ]:
# products and targets
df['product'].value_counts()

In [ ]:
df.groupby('product')['target'].value_counts(normalize = True)

Let's look at the range of time-stamps on these data:

In [ ]:
# Date range
# df['time2'] = pd.to_datetime(df['time'] )

In [24]:
df.time.min()

'2019-01-01 00:00:25'

In [25]:
df.time.max()

'2019-05-31 23:59:21'

Let's check the counts of the different site_version values:

In [ ]:
# Site types
df['site_version'].value_counts()

In [ ]:
# action title
df['title'].value_counts()

In [ ]:
# double check, Is every 'order' a true order
len(df[df['title'] == 'order'])

In [ ]:
sum(df.target)

## Experimental Setup

We need to filter by site_version, time, and product:

In [26]:
f1 = (df['site_version'] == 'desktop')
f2 = ((df['product'] == 'sneakers') | (df['product'] == 'accessories'))
f3 = (df['time'] >= '2019-05-01')

df_AB = df[f1 & f2 & f3]
df_AB

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target,time2
32,8064199,f909eb9bbf2795337dd38323b7951705,5a96585c3db94eb2ae40e559cbd2e164,94b6ea2316b773afa20925d9b79d5bf8,accessories,desktop,2019-05-19 09:26:29,banner_show,0,2019-05-19 09:26:29
38,7560076,97808cfd3d44fa7e8cfeb71bb5f02590,1e2db5d01051fc65fffc6c27f34400a3,48c8e3f287d814493cff887ff445f535,sneakers,desktop,2019-05-03 05:34:46,banner_show,0,2019-05-03 05:34:46
59,5923209,71c39cc2e2ede95a7ca296d4e3e56f85,46a6f13750d5032f366349e3e3171678,30559d0b23980e7a778c93c52ba5deca,sneakers,desktop,2019-05-19 05:04:32,banner_show,0,2019-05-19 05:04:32
105,8161379,e969a575bcf5b10d6676c2fab9cbf784,aa77bca62e9a84f5cd8d734c70afd060,70ed6e66458fda7a3164e2633d81ea07,sneakers,desktop,2019-05-29 17:51:46,banner_show,0,2019-05-29 17:51:46
115,4564531,3068b616fac2838de6a2d33a6fad60ad,aec8ee5450418b5bc97127c2269058ca,14cbe8cf92c259f5d18310347e3cb5cb,sneakers,desktop,2019-05-04 09:33:44,banner_show,0,2019-05-04 09:33:44
...,...,...,...,...,...,...,...,...,...,...
999904,5738290,ecfe9da2dbea255f05b2f526a4c663a7,cd7598ba5798ad6bd3f5a667ae4c217a,644ad50dd900d6735d8ceb15b71d50f8,accessories,desktop,2019-05-01 23:01:05,banner_show,0,2019-05-01 23:01:05
999908,7885931,8c28b45ff7fbb5d5df0b7670183689db,2f28106a4db2babcfde1c4095c805632,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-26 11:19:09,order,1,2019-05-26 11:19:09
999915,8099155,c613db2f19c7285f101196062c178490,97473368941b89c4716c4f389e9b8bd7,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-11 09:04:46,order,1,2019-05-11 09:04:46
999930,6690689,6f7f32ecf6790e4eb06384b479f43162,73f403b4843c70c41c0db294ee479582,13f395d7c86ef276f7ce52a557fd3491,accessories,desktop,2019-05-09 09:17:24,banner_show,0,2019-05-09 09:17:24


In [27]:
df_AB['site_version'].value_counts()

desktop    25972
Name: site_version, dtype: int64

In [28]:
df_AB['product'].value_counts()

sneakers       13269
accessories    12703
Name: product, dtype: int64

In [29]:
df_AB['time'].describe()

count                   25972
unique                  25806
top       2019-05-12 18:42:33
freq                        3
Name: time, dtype: object

In [30]:
# sniff test!

### What Test Would Make Sense?

Since we're comparing the frequency of conversions of customers who saw the "sneakers" banner against those who saw the "accessories" banner, we can use a $\chi^2$ test.

Note there are other hypothesis tests we can use but this should be fine since it should fit our criteria.

### The Hypotheses

$H_0$: Sneakers no more or less likely to buy than accessories

$H_1$: Sneakers are more or less likely / difference between the two

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

## $\chi^2$ Test

### Setup the Data

We need our contingency table: the numbers of people who did or did not submit orders, both for the accessories banner and the sneakers banner. 

In [31]:
# Create the two product types
df_acc = df_AB[df_AB['product'] == 'accessories']
df_sn = df_AB[df_AB['product'] == 'sneakers']

In [32]:
# Get number of orders
acc_ord = sum(df_acc['target'])
sn_ord = sum(df_sn['target'])
acc_ord,sn_ord

(496, 799)

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [34]:
# get conversion rates
acc_no_ord = len(df_acc) - acc_ord
sn_no_ord  = len(df_sn) - sn_ord
acc_no_ord,sn_no_ord

(12207, 12470)

In [42]:
# Create table
contingency_table = np.array(
[
[acc_ord,acc_no_ord],
[sn_ord,sn_no_ord]
]
)

### Calculation


In [43]:
# Chi^2!
stats.chi2_contingency(contingency_table)

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

## Interpretation

In [44]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [45]:
# Find the difference in conversion rate
accessory_CR, sneaker_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

In [46]:
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR-accessory_CR):.3f}%')

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	6.022%

Absolute difference of CR: 2.117%



So we can say:
- There was a statistically significant difference at the $\alpha$-level (confidence level)
- The difference was about $2.1\%$ in favor of the sneaker banner!

# Exercise

> The company is impressed with what you found and is now wondering if there is a difference in their other banner ads!

With your group, look at the same month (May 2019) but compare different platforms ('mobile' vs 'desktop') and or different banner types ('accessories', 'sneakers', 'clothes', 'sports_nutrition'). Just don't repeat the same test we did above 😉

Make sure you record what considerations you have for the experiment, what hypothesis test you performed ($H_0$ and $H_1$ too), and your overall conclusion/interpretation for the _business stakeholders_. Is there a follow-up you'd suggest? 

In [47]:
df['product'].value_counts()

clothes             210996
company             203020
sneakers            201298
sports_nutrition    193200
accessories         191486
Name: product, dtype: int64

In [57]:
f1 = (df['site_version'] == 'mobile')
f2 = ((df['product'] == 'sports_nutrition') | 
      (df['product'] == 'clothes'))
f3 = (df['time'] >= '2019-05-01')

df_AB = df[f1 & f2 & f3]
df_AB.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target,time2
34,6365447,ae031dd8b5684d4a13e12f9fbca8ca6e,0e2135ead33f549ba2b774f6422c09b5,637a5f326a15510ca050600d47541060,sports_nutrition,mobile,2019-05-20 17:39:52,banner_show,0,2019-05-20 17:39:52
40,5971896,19bda32538714d4e515e1f2209fd1298,88c2c9e57454f3f7e088c98442f64b58,3fc1a6f8f9d1eb67ef2f242d136923c9,sports_nutrition,mobile,2019-05-16 23:17:18,banner_show,0,2019-05-16 23:17:18
52,8379527,16588be9bc5d4ef211d476e2fbadecc0,b77361f8d06dd11a741b5cd01ac23240,2b1fe1e4b63d6aadf4c0cf6df57b63a3,clothes,mobile,2019-05-22 06:35:18,banner_show,0,2019-05-22 06:35:18
90,4557894,7d5030b43301e95bfea4e385adb3c743,f762f683aed8ab7da658d3327c0ad5f5,1f9783835a15e665aff7e554ea10345e,clothes,mobile,2019-05-08 06:37:21,banner_show,0,2019-05-08 06:37:21
92,8260648,51710dbb886eb764cc3b41cf4f1aa461,098cee69703432819be7a116c831171c,05547166b01902fa7456641630405846,clothes,mobile,2019-05-31 16:38:01,banner_show,0,2019-05-31 16:38:01


In [58]:
df_sp = df_AB[df_AB['product'] == 'sports_nutrition']
df_cl = df_AB[df_AB['product'] == 'clothes']

In [67]:
sp_ord = sum(df_sp.target)
cl_ord = sum(df_cl.target)
sp_no_order = df_sp.shape[0] - sp_ord
cl_no_order = df_cl.shape[0] - cl_ord

cont_table = np.array([[sp_ord,sp_no_order],
                       [cl_ord,cl_no_order]])
cont_table

array([[  303, 37035],
       [ 1087, 37286]])

In [68]:
stats.chi2_contingency(cont_table)

(427.8567678251773,
 4.7573490826096015e-95,
 1,
 array([[  685.49906883, 36652.50093117],
        [  704.50093117, 37668.49906883]]))

In [69]:
df['site_version'].value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

In [70]:
# Find the difference in conversion rate
sports_CR, clothes_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

print(f'Conversion Rate for sports nutrition banner:\n\t{100*sports_CR:.3f}%')
print(f'Conversion Rate for clothes banner:\n\t{100*clothes_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(clothes_CR-sports_CR):.3f}%')

Conversion Rate for sports nutrition banner:
	3.905%
Conversion Rate for clothes banner:
	6.022%

Absolute difference of CR: 2.117%
